In [1]:
import os
import pymysql
import pandas as pd
import numpy as np
from dotenv import load_dotenv, find_dotenv

In [2]:
from customerclustering.db_connection import Db
from customerclustering.get_training_data import *

In [44]:
# df0=GetTrainingData(conn=Db.db_conn(),rows=20).get_training_data()

# access to the data
# conn = pymysql.connect(
#     host=os.getenv('HOST'),
#     port=int(3306),
#     user=os.getenv('USER_DB'),
#     passwd=os.getenv('PASSWORD_DB'),
#     db=os.getenv('DB'),
#     charset='utf8mb4')

# rows=20000
# df=GetTrainingData(conn,rows).get_training_data()
# df_copy=df
# df.head()

In [45]:
training_data = pd.read_csv("../raw_data/training_data.csv")

# Categorical Features

In [82]:
import io
buffer = io.StringIO()
training_data.info(buf=buffer)
lines = buffer.getvalue().splitlines()
column_df = (pd.DataFrame([x.split() for x in lines[6:-2]], columns=lines[3].split())
       .drop('Count',axis=1)
       .rename(columns={'Non-Null':'Non-Null Count'}))
column_df.head()

,#,Column,Non-Null Count,Dtype
0,1,Product,6129,object
1,2,Status,6129,object
2,3,userID,6129,object
3,4,stripeCustID,6129,object
4,5,num_subs,6129,int64


In [87]:
#categorical data only

cat_columns = column_df[column_df['Dtype']=='object'].reset_index().Column

cat_columns_df = training_data[cat_columns].drop(columns =['userID','stripeCustID','pProfileID',\
                                                           'startDate','endDate','createDate'])
unique_value_list = []
for x in range(0, len(cat_columns_df.columns)):
    y = cat_columns_df[cat_columns_df.columns[x]].nunique()
    unique_value_list.append([cat_columns_df.columns[x], y])
    
unique_value_df = pd.DataFrame(unique_value_list, columns = ['column_name','unique_count'])
unique_value_df.sort_values(by ='unique_count')

,column_name,unique_count
8,autonomy,3
13,plan_type,3
9,access,4
1,Status,5
3,located,5
10,country,5
0,Product,6
7,complex,7
11,favActivityType,12
12,2ndFavActivityType,13


In [106]:
#Ordinal Encoding
#autonomy;
#plan_type;
#access
#Status: clean to 0 and 1, active and inactive;
#complex: 0-3; further cleaning required to get the consistent wording;


In [107]:
#OneHotEncoder
#located: clean and condense to rural and city only;
#favActivityType: reduced to 5 and other;
#2ndFavActivityType: reduced to 5 and other;


In [108]:
#feature to remove
#country: 6123 out of 6129 are from AUS;
#Product: unsure 5,772 Ausmed subscriptons vs 351 loyalty subscription; futher cleaning needed;


## feature reviewing

In [88]:
training_data['autonomy'].value_counts()

moderate    3848
high        1632
minimal      649
Name: autonomy, dtype: int64

In [90]:
training_data['plan_type'].value_counts()

quarterly    4458
monthly      1012
annually      659
Name: plan_type, dtype: int64

In [92]:
training_data['access'].value_counts()

always       3634
usually      1928
sometimes     536
never          31
Name: access, dtype: int64

In [93]:
training_data['Status'].value_counts()

active                3280
canceled              2713
past_due               112
incomplete_expired      21
trialing                 3
Name: Status, dtype: int64

In [94]:
training_data['located'].value_counts()

metropolitan centre    2586
large rural centre     1350
capital city           1118
small rural centre      832
remote area             243
Name: located, dtype: int64

In [95]:
training_data['country'].value_counts()

AUS    6123
NZL       3
ARE       1
GBR       1
ZAF       1
Name: country, dtype: int64

In [96]:
training_data['Product'].value_counts()

Ausmed Subscription            5252
Ausmed Subscription Monthly     520
Loyalty Subscription            271
Ausmed Loyalty Monthly           80
Ausmed US Subscription            4
Ausmed NZ Subscription            2
Name: Product, dtype: int64

In [97]:
training_data['complex'].value_counts()

generally complex         3635
very complex              1332
high complexity            733
low complexity             426
have a high complexity       1
be generally complex         1
be very complex              1
Name: complex, dtype: int64

In [101]:
training_data['favActivityType'].value_counts()

e-Learning           3415
Textbook             1360
Competency            517
In-Service            405
Conference            138
Other                 129
Meeting                45
Teaching               42
Mentoring              39
Postgraduate           21
Procedure              15
Peer Consultation       3
Name: favActivityType, dtype: int64

In [102]:
training_data['2ndFavActivityType'].value_counts()

e-Learning           2609
Textbook             1635
Competency            628
In-Service            552
Other                 239
Conference            222
Teaching               76
Meeting                62
Mentoring              53
Postgraduate           29
Procedure              21
Peer Consultation       2
Research                1
Name: 2ndFavActivityType, dtype: int64